In [5]:
from rtf_utils.moat_utils import MoatTile
import json
import logging
import sys
import os
from rtf_utils.gcp_utils import CloudStorage,BigQuery
from essence.analytics.platform import securedcredentials as secure_creds

In [6]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'
credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)


moat_token = "[moat token here]"
start_date = '20190801'
end_date = '20190810'

C:\Users\kyle.randolph\AppData\Local\Continuum\anaconda3\lib\site-packages\google\auth\_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
ig_vid = MoatTile(tile_id=13120,
         tile_type="vid",name = "ig_vid",
         level_filters={'level1':10154328017481183},
         dimensions=['date','level4'])

yt_trv = MoatTile(tile_id=13386,
         tile_type="vid",name = "YT_TRV",
         level_filters={'level1':7020493427},
         dimensions=['date','level4'])

cm_disp = MoatTile(tile_id=2506,
         tile_type="disp",name = "cm_disp",
         level_filters={'level1':22830945},
         dimensions=['date','level3'])
                   
cm_vid = MoatTile(tile_id=2698,
         tile_type="vid",name = "cm_vid",
         level_filters={'level1':22830945},
         dimensions=['date','level3'])


file_queue = [ig_vid,yt_trv]

In [ ]:
files = [tile.get_data('20190801', '20190805',moat_token) for tile in file_queue]    

In [8]:
gcs = CloudStorage(credentialsFromVault)
bq = BigQuery(credentialsFromVault)

In [23]:
for file in files:
    print("uploading {}".format(file))
    gcs.upload_blob('rtf_staging','bonito_q3/'+file,file,mode='filename')
    try:
        print("Removing {} Locally".format(file))
        os.remove(file)
    except Exception as e:
        print(e)

uploading 13120_ig_vid.json
uploading 13386_YT_TRV.json


In [29]:
files

['13120_ig_vid.json', '13386_YT_TRV.json']

In [11]:
bq.load_from_gcs('rtf_brand_reporting','gs://rtf_staging/bonito_q3/13386_YT_TRV.json','test_write_trv',mode='Truncate',ext='json')

## ------ DCM Section ------

#### DCM ToDo:
- refactor report class take same data range format. Declare one format globally and format on class instantiation
- Put dcm report cleaner in class
- put report wait-until-available in class
- figure out a better way to materalize views -- storing sql etc (use parameters for tables names

In [13]:
from rtf_utils.dfa_utils import CampaignManagerReport,clean_dcm_file

In [ ]:
verification_report = CampaignManagerReport(credentialsFromVault,5096586,631165400)
video_player_size = CampaignManagerReport(credentialsFromVault,5096586,632709822)

verification_report.set_date_range("2019-07-01","2019-08-25")
video_player_size.set_date_range("2019-07-01","2019-08-25")

verification_report.run_report()
video_player_size.run_report()

verification_report.get_report_status()


file_name = verification_report.download_file()


from time import sleep
video_player_size.get_report_status()
while video_player_size.status != "REPORT_AVAILABLE":
    sleep(5)
    video_player_size.get_report_status()
    print(video_player_size.status)
filename = video_player_size.download_file()
clean_dcm_file('Bonito_Video_Player_Size.csv')

In [ ]:
def clean_dcm_file(filename):
    data = []
    write = False
    with open(filename,'r') as f:
        reader = csv.reader(f, delimiter=',')

        for row in reader:
            if write == True:
                data.append(row)        
            elif row == ['Report Fields']:
                write = True

        if data[-1][0] == 'Grand Total:':
            data.pop()

    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

In [ ]:
from time import sleep
video_player_size.get_report_status()
while video_player_size.status != "REPORT_AVAILABLE":
    sleep(5)
    video_player_size.get_report_status()
    print(video_player_size.status)
filename = video_player_size.download_file()
clean_dcm_file('Bonito_Video_Player_Size.csv')

In [ ]:
materialize_verification = """
                            SELECT
                              date,
                              CAST(REGEXP_EXTRACT(placement,r"OPID-(\d+)") AS int64) AS opid,
                              SUM(verifiable_impressions) verifiable_impressions_no_issues
                            FROM
                              `essence-analytics-dwh.rtf_brand_reporting.DCM_contentVerification`
                            WHERE
                              classifier = "None Detected"
                            GROUP BY
                              1,
                              2

                            """

materialize_player_report = """
                            WITH
                              player_pivot AS (
                              SELECT
                                date,
                                CAST(REGEXP_EXTRACT(placement,r"OPID-(\d+)") AS int64) AS opid,
                                CASE
                                  WHEN video_player_size = "HD" THEN verifiable_impressions
                              END
                                AS hd_player_impressions,
                                CASE
                                  WHEN video_player_size = "LARGE" THEN verifiable_impressions
                              END
                                AS large_player_impressions,
                                CASE
                                  WHEN video_player_size = "SMALL" THEN verifiable_impressions
                              END
                                AS small_player_impressions,
                                CASE
                                  WHEN video_player_size = "(not set)" THEN verifiable_impressions
                              END
                                AS player_not_set_impressions
                              FROM
                                `essence-analytics-dwh.rtf_brand_reporting.DCM_playerSize`)
                            SELECT
                              date,
                              opid,
                              SUM(hd_player_impressions) hd_player_impressions,
                              SUM(large_player_impressions) large_player_impressions,
                              SUM(small_player_impressions) small_player_impressions,
                              SUM(player_not_set_impressions) player_not_set_impressions
                            FROM
                              player_pivot
                            GROUP BY
                              1,
                              2
                            """


moat_vid = """
            SELECT
              date,
              CAST(REGEXP_EXTRACT(level4_label,r"OPID-(\d+)") AS int64) AS opid,
              SUM(impressions_analyzed) AS moat_vid_impressions_analyzed,
              SUM(susp_valid) AS moat_vid_susp_valid,
              SUM(valid_and_viewable) AS valid_and_viewable,
              SUM(reached_first_quart_sum) reached_first_quart_sum,
              SUM(reached_second_quart_sum) reached_second_quart_sum,
              SUM(reached_third_quart_sum) reached_third_quart_sum,
              SUM(reached_complete_sum) reached_complete_sum,
              SUM(player_visible_on_complete_sum) player_visible_on_complete_sum,
              SUM(player_audible_on_complete_sum) player_audible_on_complete_sum,
              SUM(player_vis_and_aud_on_complete_sum) player_vis_and_aud_on_complete_sum,
              SUM(susp_valid_and_inview_gm_meas_sum) susp_valid_and_inview_gm_meas_sum,
              sum(_5_sec_in_view_impressions) _5_sec_in_view_impressions,
              sum(susp_bot_geo_perc) susp_bot_geo_perc
            FROM
              `essence-analytics-dwh.rtf_brand_reporting.MOAT_videoTiles_RAW`

            Group by 1,2
            """

helper_sql.moat_disp = """
            SELECT
              date,
              CAST(REGEXP_EXTRACT(level3_label,r"OPID-(\d+)") AS int64) AS opid,
              SUM(impressions_analyzed) AS impressions_analyzed,
              SUM(valid_and_viewable) AS valid_and_viewable,
              SUM(iva) AS iva,
              SUM(susp_bot_geo_perc) AS susp_bot_geo_perc
            FROM
              `essence-analytics-dwh.rtf_brand_reporting.MOAT_displayTiles_RAW`
            GROUP BY
              1,
              2
            """



In [ ]:
resp = bq.store_query_results(moat_disp,"rtf_brand_reporting","MOAT_displayTiles_TRANSFORMED")